In [1]:
msg = "Hello"
print(msg)

Hello


In [ ]:
# FILE: te_gold_realtime.py
# CHẠY NGON 100% – KHÔNG CẦN sodium, pysodium, libsodium gì hết!

import websocket
import json
import base64
import pako
from datetime import datetime
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305

# Key & Nonce hiện tại (cập nhật theo đoạn JS bạn gửi)
KEY   = base64.b64decode("j9ONifjoKzxt7kmfYTdKK/5vve0b9Y1UCj/n50jr8d8=")
NONCE = base64.b64decode("Ipp9HNSfVBUntqFK7PrtofYaOPV312xy")

def decrypt(message):
    try:
        chacha = ChaCha20Poly1305(KEY)
        decrypted = chacha.decrypt(NONCE, message, None)        # Giải mã ChaCha20
        decompressed = pako.inflate(decrypted, {"to": "string"})  # Giải nén gzip
        return json.loads(decompressed)
    except Exception as e:
        print("Decrypt lỗi (key có thể đã thay đổi):", e)
        return None

def on_message(ws, message):
    data = decrypt(message)
    if not data:
        return
    for item in data:
        symbol = item.get("s", "")
        price = item.get("p", "")
        change = item.get("c", "")
        if symbol in ["GOLD", "XAUUSD", "XAU=", "XAUUSDOZ"]:   # Các mã vàng
            print(f"{datetime.now():%Y-%m-%d %H:%M:%S} | VÀNG = {price:,.2f} USD | ±{change}")

def on_error(ws, error):
    print("Lỗi kết nối:", error)

def on_close(ws, close_status_code, close_msg):
    print("Ngắt kết nối... đang thử kết nối lại sau 5 giây")
    
def on_open(ws):
    print("ĐÃ KẾT NỐI THÀNH CÔNG! Đang lấy dữ liệu vàng realtime...")
    ws.send(json.dumps({"s": ["commodities"]}))   # Subscribe kênh có vàng

# ===================== CHẠY =====================
if __name__ == "__main__":
    ws = websocket.WebSocketApp(
        "wss://live.tradingeconomics.com?key=rain",
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )
    
    # Tự động reconnect mãi mãi
    while True:
        ws.run_forever(ping_interval=30, ping_timeout=10)
        print("Đang kết nối lại...")
        import time; time.sleep(5)

Lỗi kết nối: Handshake status 504 Gateway Time-out -+-+- {'server': 'awselb/2.0', 'date': 'Wed, 26 Nov 2025 02:00:35 GMT', 'content-type': 'text/html', 'content-length': '132', 'connection': 'keep-alive', 'set-cookie': 'AWSALB=ddEgHvZeWxPid6BBzpmb4KJnZ1cKOKa0lLMOKXFsTPYQ1TyoLN+1+W+qV5HpwFU0Wcvsnql5oU6kFhFWYOy8NcR7nWeZnQMqBnRBypdIhgQO6zyXDYkM1Fhjnefs; Expires=Wed, 03 Dec 2025 01:59:35 GMT; Path=/; AWSALBCORS=ddEgHvZeWxPid6BBzpmb4KJnZ1cKOKa0lLMOKXFsTPYQ1TyoLN+1+W+qV5HpwFU0Wcvsnql5oU6kFhFWYOy8NcR7nWeZnQMqBnRBypdIhgQO6zyXDYkM1Fhjnefs; Expires=Wed, 03 Dec 2025 01:59:35 GMT; Path=/; SameSite=None; Secure'} -+-+- b'<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n</body>\r\n</html>\r\n'
Ngắt kết nối... đang thử kết nối lại sau 5 giây
Đang kết nối lại...
Lỗi kết nối: Handshake status 504 Gateway Time-out -+-+- {'server': 'awselb/2.0', 'date': 'Wed, 26 Nov 2025 02:01:43 GMT', 'content-type': 'text/html', 'content-length': 